# Make dataset for next vertex prediction

#### Using the same scan splits as seed 42 in original current room prediction

In [4]:
import sys
import os
import multiprocessing
import random
import numpy as np
import math
import torch
from torch.utils.data import Dataset, DataLoader
# from tensorboardX import SummaryWriter

import utils

In [29]:
try:
    sys.path.append('/opt/MatterSim/build/')  # local docker or Philly
    import MatterSim
except: 
    # local conda env only
    sys.path.append('/home/hoyeung/Documents/vnla/code/build')  
    import MatterSim

In [8]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_room_types.txt', "r") as f:
    room_types = f.read().split('\n')[:-1]
len(room_types)

30

In [10]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_tr_scans.txt', "r") as f:
    scans = f.read().split('\n')[:-1]
len(scans)

56

In [2]:
def read_indices_and_labels(path):
    feature_ids = []
    labels = []
    with open(path, 'r') as f:
        lines = f.read().split('\n')[:-1]
        print ("Number of lines = {}".format(len(lines)))
        for line in lines:
            long_id, viewix, label = line.split('\t')
            feature_ids.append((long_id, viewix))
            labels.append(label)
    return feature_ids, labels

In [124]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
filename = 'train_indices.txt'
train_indices, train_curr_rm_labels = read_indices_and_labels(os.path.join(exp_dir, filename))
train_indices[100:105]

Number of lines = 224316


[('5LpN3gDmAk7_c75f71faa8b74a8eb875731b3dbe6183', '3'),
 ('5LpN3gDmAk7_c75f71faa8b74a8eb875731b3dbe6183', '4'),
 ('5LpN3gDmAk7_c75f71faa8b74a8eb875731b3dbe6183', '5'),
 ('5LpN3gDmAk7_c75f71faa8b74a8eb875731b3dbe6183', '6'),
 ('5LpN3gDmAk7_c75f71faa8b74a8eb875731b3dbe6183', '7')]

In [25]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
filename = 'val_seen_indices.txt'
val_seen_indices, val_seen_curr_rm_labels = read_indices_and_labels(os.path.join(exp_dir, filename))
val_seen_indices[100:105]

Number of lines = 18540


[('5LpN3gDmAk7_58351a02ce7742bd9a6674db0131a534', '6'),
 ('5LpN3gDmAk7_58351a02ce7742bd9a6674db0131a534', '8'),
 ('5LpN3gDmAk7_58351a02ce7742bd9a6674db0131a534', '21'),
 ('5LpN3gDmAk7_58351a02ce7742bd9a6674db0131a534', '33'),
 ('5LpN3gDmAk7_bc48486938fb46c1b18c3a30767283c5', '4')]

In [7]:
exp_dir = '/home/hoyeung/blob_experiments/output_philly/20200202_philly_debug_semantics/classifier_seed_42_batch_size_100_lr_0.001_layers_2/'
filename = 'val_unseen_indices.txt'
val_unseen_indices, val_unseen_curr_rm_labels = read_indices_and_labels(os.path.join(exp_dir, filename))
val_unseen_indices[100:105]

Number of lines = 18540


[('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '28'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '29'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '30'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '31'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '32')]

In [5]:
exp_dir = '/home/hoyeung/blob_experiments/output_local/20200203_debug_semantics/debug_local_single_frame/'
filename = 'val_unseen_indices.txt'
val_unseen_indices, val_unseen_curr_rm_labels = read_indices_and_labels(os.path.join(exp_dir, filename))
val_unseen_indices[100:105]

Number of lines = 4248


[('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '28'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '29'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '30'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '31'),
 ('rPc6DW4iMge_2d20ebd11b9d4855bac237d15181e126', '32')]

## Start a sim

In [70]:
def _check_center(closest_vertex):
    '''within 30 deg horizontally and vertically)'''
    if (closest_vertex.rel_heading < math.pi/12.0 and closest_vertex.rel_heading > -math.pi/12.0) and \
    (closest_vertex.rel_elevation < math.pi/12.0 and closest_vertex.rel_elevation > -math.pi/12.0):
        return True

def _check_above(closest_vertex):
    '''within 30 deg horizontally and more than 15 deg above'''
    if (closest_vertex.rel_heading < math.pi/12.0 and closest_vertex.rel_heading > -math.pi/12.0) and \
    (closest_vertex.rel_elevation > math.pi/12.0):
        return True

def _check_below(closest_vertex):
    '''within 30 deg horizontally and more than 15 deg below'''
    if (closest_vertex.rel_heading < math.pi/12.0 and closest_vertex.rel_heading > -math.pi/12.0) and \
    (closest_vertex.rel_elevation < -math.pi/12.0):
        return True

In [68]:
os.environ['PT_DATA_DIR'] ='/home/hoyeung/blob_matterport3d/'

In [33]:
PT_DATA_DIR = '/home/hoyeung/blob_matterport3d/'
sim = MatterSim.Simulator()
sim.setRenderingEnabled(False)
sim.setDiscretizedViewingAngles(True)
sim.setCameraResolution(640, 480)
sim.setCameraVFOV(math.radians(60))
sim.setNavGraphPath(
    os.path.join(PT_DATA_DIR, 'connectivity'))
sim.init()

In [60]:
viewix_heading_elevation_map = {}
for i in range(0,12):
    viewix_heading_elevation_map[i] = (i*math.pi/6, -math.pi/6)
for i in range(12,24):
    viewix_heading_elevation_map[i] = (i*math.pi/6, 0)
for i in range(24,36):
    viewix_heading_elevation_map[i] = (i*math.pi/6, +math.pi/6)

In [102]:
def filter_with_available_next_vertex(indices):
    valid_indices = []
    next_rm_labels = []
    curr_scanId = indices[0][0].split('_')[0]
    scan_panos_to_region = utils.load_panos_to_region(curr_scanId,"")

    for index in indices:

        long_id, viewix = index[0], int(index[1])
        scanId, viewpointId = long_id.split('_')

        if curr_scanId != scanId:
            curr_scanId = scanId
            scan_panos_to_region = utils.load_panos_to_region(curr_scanId,"")

        heading, elevation = viewix_heading_elevation_map[viewix]
        sim.newEpisode(scanId, viewpointId, heading, elevation)
        state = sim.getState()
        assert state.viewIndex == viewix

        # map to direct vertex!
        if len(state.navigableLocations) > 1:
            closest_vertex = state.navigableLocations[1]
            # determine of we are facing the vertex directly
            if _check_center(closest_vertex) or (viewix // 12 == 2 and _check_above(closest_vertex)) or (viewix // 12 == 0 and _check_below(closest_vertex)):
                next_vertexId = closest_vertex.viewpointId
                next_rm_labels.append(scan_panos_to_region[next_vertexId])
                valid_indices.append(index)
    assert len(valid_indices) == len(next_rm_labels)
    print ('original input length = {}'.format(len(indices)))
    print ('filtered input length = {}'.format(len(valid_indices)))
    return valid_indices, next_rm_labels

In [103]:
val_unseen_valid_indices, val_unseen_next_rm_labels = filter_with_available_next_vertex(val_unseen_indices)

original input length = 18540
filtered input length = 1788


In [105]:
val_seen_valid_indices, val_seen_next_rm_labels = filter_with_available_next_vertex(val_seen_indices)

original input length = 18540
filtered input length = 1977


In [136]:
train_valid_indices, train_next_rm_labels = filter_with_available_next_vertex(train_indices)

original input length = 224316
filtered input length = 23046


In [137]:
train_next_rm_labels[:10]

['p', 'p', 'p', 'p', 'p', 'p', 'k', '-', 'p', 'p']

## Write the indices out

In [138]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_train_next_vertex_indices.txt', 'w') as fh:
    for feat_id, rm_label in zip(train_valid_indices, train_next_rm_labels):
        fh.write("{}\t{}\t{}\n".format(feat_id[0], feat_id[1], rm_label))

In [139]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_val_seen_next_vertex_indices.txt', 'w') as fh:
    for feat_id, rm_label in zip(val_seen_valid_indices, val_seen_next_rm_labels):
        fh.write("{}\t{}\t{}\n".format(feat_id[0], feat_id[1], rm_label))

In [140]:
with open('/home/hoyeung/blob_matterport3d/semantics/asknav_val_unseen_next_vertex_indices.txt', 'w') as fh:
    for feat_id, rm_label in zip(val_unseen_valid_indices, val_unseen_next_rm_labels):
        fh.write("{}\t{}\t{}\n".format(feat_id[0], feat_id[1], rm_label))

In [4]:
def read_indices_and_labels(path):
    feature_ids = []
    labels = []
    with open(path, 'r') as f:
        lines = f.read().split('\n')[:-1]
        print ("Number of lines = {}".format(len(lines)))
        for line in lines:
            long_id, viewix, label = line.split('\t')
            feature_ids.append((long_id, viewix))
            labels.append(label)
    return feature_ids, labels

In [146]:
with open(filename, 'r') as fh:
    data = fh.read().split('\n')
data

['5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf\t18\tp',
 '5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf\t20\tp',
 '5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf\t21\tp',
 '5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf\t22\tp',
 '5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf\t23\tp',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t12\tp',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t19\tk',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t20\t-',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t21\tp',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t22\tp',
 '5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c\t23\tp',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t12\tp',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t14\tp',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t18\tf',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t19\t-',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t20\tp',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t22\tp',
 '5LpN3gDmAk7_7dca42b9f8a04177b4766e6b90a76aef\t

In [142]:
train_valid_indices[:10]

[('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '18'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '20'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '21'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '22'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '23'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '12'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '19'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '20'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '21'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '22')]

In [5]:
filename = '/home/hoyeung/blob_matterport3d/semantics/asknav_train_next_vertex_indices.txt'
train_load_indices, train_load_next_rm_labels = read_indices_and_labels(filename)
train_load_indices[:10]

Number of lines = 23046


[('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '18'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '20'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '21'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '22'),
 ('5LpN3gDmAk7_bda8025f20404048a77381e9e0dc0ccf', '23'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '12'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '19'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '20'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '21'),
 ('5LpN3gDmAk7_69c059f0f5c6450e8cc5e6459bd4cb3c', '22')]

In [144]:
train_load_next_rm_labels[:10]

'S'